In [50]:
!pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [51]:
pip install --upgrade pip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [52]:
import langchain
import groq
import langchain_groq
import langchain_openai
import chromadb
import streamlit
import PyPDF2
import dotenv
import sentence_transformers
import os
import langchain_community
import openai
import tiktoken
import langchain_chroma

In [53]:
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("GROQ_API_KEY")
embedding_api_key=os.getenv("OPENAI_API_KEY")
hf_api_key=os.getenv("HF_API_KEY")


In [54]:
pdf_path = r"/Users/tasmiyataj/Downloads/CS_HDC_2 - Copy/docs/Heart-Disease-Classifications-and-Subclasses.pdf"

if os.path.exists(pdf_path):
    print(f"PDF file found at {pdf_path}")
else:
    print(f"PDF file not found at {pdf_path}")

PDF file found at /Users/tasmiyataj/Downloads/CS_HDC_2 - Copy/docs/Heart-Disease-Classifications-and-Subclasses.pdf


In [55]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma 
from PyPDF2 import PdfReader
from langchain.docstore.document import Document

def load_pdf_with_pypdf2(file_path):
    loader = PdfReader(file_path)
    documents = []
    for page_num, page in enumerate(loader.pages):
        text = page.extract_text()
        if text:
            doc = Document(page_content=text, metadata={"page": page_num})
            documents.append(doc)
    return documents

documents=load_pdf_with_pypdf2(pdf_path)

In [56]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=768, chunk_overlap=256)
documents = text_splitter.split_documents(documents)
documents

[Document(metadata={'page': 0}, page_content='Heart\n \nDisease\n \nClassiﬁcations\n \nand\n \nSubclasses\n \n \n \nSummary:\n \n●\n \nResearch\n \nsuggests\n \nthere\n \nare\n \naround\n \n10\n \nmain\n \nclassiﬁcations\n \nof\n \nheart\n \ndiseases,\n \nincluding\n \ncoronary\n \nartery\n \ndisease,\n \narrhythmias,\n \nand\n \nheart\n \nfailure,\n \nwith\n \nsome\n \ncontroversy\n \nover\n \ngrouping,\n \nsuch\n \nas\n \nwhether\n \nrheumatic\n \nheart\n \ndisease\n \nis\n \nseparate\n \nfrom\n \nvalvular\n \nheart\n \ndisease.\n \n●\n \nEach\n \nclassiﬁcation\n \nhas\n \nmultiple\n \nsubclasses,\n \nsuch\n \nas\n \nstable\n \nangina\n \nunder\n \ncoronary\n \nartery\n \ndisease\n \nor\n \natrial\n \nﬁbrillation\n \nunder\n \narrhythmias,\n \nwith\n \nthe\n \ntotal\n \nnumber\n \nvarying\n \nby\n \nsource\n \nbut\n \noften\n \nin\n \nthe\n \ndozens.\n \n●\n \nThe\n \nevidence\n \nleans\n \ntoward\n \nno\n \nsingle\n \nstandard\n \nclassiﬁcation,\n \nreﬂecting\n \nthe'),
 Document(me

In [57]:
def remove_newlines(documents):
    for doc in documents:
        doc.page_content = doc.page_content.replace('\n', '')
    return documents
documents = remove_newlines(documents)
documents 

[Document(metadata={'page': 0}, page_content='Heart Disease Classiﬁcations and Subclasses   Summary: ● Research suggests there are around 10 main classiﬁcations of heart diseases, including coronary artery disease, arrhythmias, and heart failure, with some controversy over grouping, such as whether rheumatic heart disease is separate from valvular heart disease. ● Each classiﬁcation has multiple subclasses, such as stable angina under coronary artery disease or atrial ﬁbrillation under arrhythmias, with the total number varying by source but often in the dozens. ● The evidence leans toward no single standard classiﬁcation, reﬂecting the'),
 Document(metadata={'page': 0}, page_content='artery disease or atrial ﬁbrillation under arrhythmias, with the total number varying by source but often in the dozens. ● The evidence leans toward no single standard classiﬁcation, reﬂecting the complexity and ongoing debate in medical literature. In summary, research suggests heart diseases are classiﬁ

In [58]:
!pip install db

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [59]:
!pip install chroma

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [60]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

# Example documents
batch = [Document(page_content="This is a test document."), Document(page_content="Another document.")]

# Embedding model
embedding_model_hf = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create vector store
db = Chroma.from_documents(batch, embedding=embedding_model_hf, persist_directory="chroma")

# Persist to disk
db.persist()


In [61]:
from langchain_groq import ChatGroq

llm = ChatGroq(api_key=api_key, 
               model_name="llama-3.3-70b-versatile", 
               temperature=0.2)

In [62]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
) 


In [63]:
#query = input("Enter your query: ")
#print("Query:", query)
#result = qa_chain.invoke({"query": query})
#print("Answer:", result["result"])


In [64]:
!pip install flask

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from langchain.chains import RetrievalQA
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
import threading
import os

# Create Flask app, telling it where to find the templates folder
app = Flask(__name__, template_folder='templates')
CORS(app)

# Assuming you have your LLM and DB set up already
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# === Route to serve chatbot.html from templates ===
@app.route('/')
def index():
    return render_template('chatbot.html')  # looks inside templates/chatbot.html

# === Endpoint for handling user messages ===
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        query = data.get('symptoms', '')
        if not query:
            return jsonify({'reply': 'Please enter some symptoms.'})

        # Get the response from the QA chain
        result = qa_chain.invoke({"query": query})
        
        # Extract answer and source documents
        answer = result.get("result", "")
        source_documents = result.get("source_documents", [])

        # Serialize source documents into a JSON-friendly format
        # Assuming Document objects have a 'text' attribute that contains the relevant content
        source_documents_serialized = [doc.text if hasattr(doc, 'text') else str(doc) for doc in source_documents]

        # Return the answer and serialized source documents
        return jsonify({
            'reply': answer,
            'source_documents': source_documents_serialized
        })
    
    except Exception as e:
        return jsonify({'reply': f"Error: {str(e)}"}), 500

# === Start the Flask server in a background thread ===
def run_flask():
    app.run(host='0.0.0.0', port=5005, debug=False, use_reloader=False)

# Launch server if not already running
flask_thread = threading.Thread(target=run_flask)
flask_thread.setDaemon(True)
flask_thread.start()

print("✅ Flask server running at http://127.0.0.1:5005")
print("👉 Visit it in your browser to interact with the chatbot.")


✅ Flask server running at http://127.0.0.1:5005
👉 Visit it in your browser to interact with the chatbot.


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5005 is in use by another program. Either identify and stop that program, or start the server with a different port.
